In [1]:
# imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option("display.max_rows", 70)

In [2]:
# importing dataset


## for desktop
df = pd.read_csv(r"..\..\sources\initial_datasets\initial_CH_POI_dataset.osm", sep = "|")



C:\Users\Margni\.virtualenvs\code_blueprint-DgG63NCU\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df["SUBCATEGORY"].unique()


array(['BUSSTOP', 'VILLAGE', 'FUEL', 'CINEMA', 'HOTEL', 'CAFE',
       'ATTRACTION', 'PARKING', 'FASTFOOD', 'STATION', 'RESTAURANT',
       'TOWN', 'CITY', 'TRAMSTOP', 'PEAK', 'PUB', 'MONUMENT',
       'FERRYTERMINAL', 'MEMORIAL', 'HAMLET', 'CHRISTIAN', 'SUBURB',
       'THEATRE', 'CAMPING', 'BAR', 'POLICE', 'POSTOFFICE', 'ATM', 'GOLF',
       'NURSERY', 'TOILETS', 'CLINIC', 'TOLLGATE', 'LIBRARY', 'MUSEUM',
       'BANK', 'SUPERMARKET', 'ALPINEHUT', 'UNIVERSITY', 'SCHOOL',
       'ARCHAELOGICAL', 'TOWNHALL', 'FIRESTATION', 'ISLAMIC', 'BUNKER',
       'CONVENIENCE', 'SWIMMING', 'CAR', 'HOSTEL', 'DEPARTMENTSTORE',
       'MILITARY', 'TOWER', 'BUDDHIST', 'FOUNTAIN', 'AIRPORT',
       'INFORMATION', 'PHARMACY', 'SOCIALCARE', 'EVENTVENUE', 'ALCOHOL',
       'CASTLE', 'KIOSK', 'HAIRDRESSER', 'JEWISH', 'LEISURECENTER',
       'COLLEGE', 'ART', 'GOVERNMENT', 'ICESKATING', 'BAKERY',
       'GUESTHOUSE', 'PUBLICBUILDING', 'SOCCER', 'BOOK', 'CLOTHES', 'DIY',
       'NIGHTCLUB', 'BICYCLE', 'MOTORR

In [4]:
df[["SUBCATEGORY"]].value_counts()

SUBCATEGORY
BUSSTOP        20815
HAMLET         16015
RESTAURANT     15376
SCHOOL          5987
PEAK            5671
               ...  
LIGHTHOUSE         2
WRECK              2
RUGBY              1
WINDMILL           1
DIVING             1
Length: 209, dtype: int64

In [5]:
# missing values count

df.isnull().sum() # no missing values for important features

ID                         0
NAME                       0
CATEGORY                   0
SUBCATEGORY                0
LON                        0
LAT                        0
SRID                       0
WKT                        0
INTERNATIONAL_NAME    178147
STREET                129986
WIKIPEDIA             172436
PHONE                 155010
CITY                  135960
EMAIL                 168087
ALTERNATIVE_NAME      169387
OPENING_HOURS         150092
DESCRIPTION           175653
WEBSITE               136792
LAST_UPDATE           178099
OPERATOR              132629
POSTCODE              131779
COUNTRY               162195
FAX                   175439
IMAGE                 177228
HOUSENUMBER           132064
OTHER_TAGS                 0
dtype: int64

In [6]:
df = df.drop(["SRID","WKT", "WIKIPEDIA", "PHONE","STREET", "LAST_UPDATE", "ID", "OPERATOR", "POSTCODE", "FAX", "IMAGE", "HOUSENUMBER", "INTERNATIONAL_NAME", "ALTERNATIVE_NAME", "EMAIL", "COUNTRY", "DESCRIPTION", "WEBSITE"], axis = 1)

In [7]:
df = df[(df["LAT"] < 90) & (df["LAT"] > -90) & (df["LON"] > -90)  & (df["LON"] < 90)]

In [8]:
# splitting the dataset in poi and hotel datasets

poi_df, hotels_df = [x for _, x in df.groupby(df['SUBCATEGORY'] == "HOTEL")]


In [9]:
poi_df

,NAME,CATEGORY,SUBCATEGORY,LON,LAT,CITY,OPENING_HOURS,OTHER_TAGS
0,Clos de l'Ombren,TRANSPORT,BUSSTOP,6.954822,46.329489,NaN,NaN,"{""bus"":""yes"",""uic_ref"":""8587746"",""name"":""Clos ..."
1,Güterbahnhof,TRANSPORT,BUSSTOP,8.519682,47.380436,NaN,NaN,"{""bus"":""yes"",""ref"":""1076"",""uic_ref"":""8591169"",..."
2,Nations,TRANSPORT,BUSSTOP,6.139638,46.222755,NaN,NaN,"{""bus"":""yes"",""uic_ref"":""8587058"",""name"":""Natio..."
3,Morgins,SETTLEMENTS,VILLAGE,6.858102,46.237659,NaN,NaN,"{""openGeoDB:loc_id"":""32130"",""openGeoDB:name"":""..."
4,Champéry,SETTLEMENTS,VILLAGE,6.868767,46.175679,NaN,NaN,"{""openGeoDB:community_identification_number"":""..."
...,...,...,...,...,...,...,...,...
178218,Parkhaus Bahnhof Sursee,AUTOMOTIVE,PARKING,8.097286,47.172344,NaN,24/7,"{""parking"":""multi-storey"",""access"":""yes"",""amen..."
178219,Résidance La Sorne,PUBLICSERVICE,SOCIALCARE,7.353392,47.367783,NaN,NaN,"{""name"":""Résidance La Sorne"",""type"":""multipoly..."
178220,Tertianum Gartenhof,PUBLICSERVICE,SOCIALCARE,8.688531,47.507789,Winterthur,NaN,"{""amenity"":""social_facility"",""type"":""multipoly..."
178221,Zona Industriale Pobia / Via Résiga,LANDUSE,INDUSTRIAL,8.996186,45.839151,NaN,NaN,"{""landuse"":""industrial"",""name"":""Zona Industria..."


In [10]:
# removing HAMLET AND BUSSTOP poi because not very useful

poi_df = poi_df[(poi_df.SUBCATEGORY != "HAMLET") & (poi_df.SUBCATEGORY != "BUSSTOP")]


In [11]:
# 
# # adding the city for every poi
# from geopy.geocoders import Nominatim
# 
# geolocator = Nominatim(user_agent = "web_service_project")
# nan_count = 0
# 
# for i in poi_df.iloc[0:10000].index:
# 
#     latitude = poi_df.loc[i, "LAT"]
#     longitude = poi_df.loc[i, "LON"]
#     location = geolocator.reverse(str(latitude) + ", " + str(longitude))
#     address = location.raw['address']
# 
#     if "city" in address:
#         poi_df.loc[i, "CITY"] = address["city"]
#     elif "town" in address:
#         poi_df.loc[i, "CITY"] = address["town"]
#     elif "village" in address:
#         poi_df.loc[i, "CITY"] = address["village"]
#     else:
#         poi_df.loc[i, "CITY"] = np.nan
#         nan_count += 1
# 
# print(nan_count)


In [12]:
poi_df["countryName"] = "Switzerland"

C:\Users\Margni\AppData\Local\Temp/ipykernel_4552/910059090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_df["countryName"] = "Switzerland"


In [13]:
poi_df = poi_df.rename(columns={'NAME': 'poiName', 'CATEGORY': 'category', 'SUBCATEGORY': 'subcategory', "LAT": "latitude",
                                'LON': 'longitude', 'OPENING_HOURS': 'openingHours','CITY': 'cityName', 'OTHER_TAGS': "otherTags"})

In [14]:
hotels_df.to_csv("../../sources/initial_datasets/hotels_df_from_poi.csv")

poi_df.to_csv("../../sources/finite_datasets/poi_dataset.csv")